In [1]:
!rm -f watson_streaming_pipelines.py*
!wget https://raw.githubusercontent.com/Yura32000/practicals/master/watson_streaming_pipelines.py

--2017-12-04 07:20:01--  https://raw.githubusercontent.com/Yura32000/practicals/master/watson_streaming_pipelines.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8016 (7.8K) [text/plain]
Saving to: ‘watson_streaming_pipelines.py’

100%[======================================>] 8,016       --.-K/s   in 0s      

2017-12-04 07:20:02 (22.2 MB/s) - ‘watson_streaming_pipelines.py’ saved [8016/8016]



In [2]:
# @hidden_cell
# The Object Storage credentials should NOT come from the 'Service credentials' page.
# Get them from the option in DSX Notebook to 'insert to code' from the Data -> Connections.
# Then add an entry for the service's PUBLIC endpoint.

cos_credentials_stage1_wd = {
  'iam_url':'https://iam.stage1.ng.bluemix.net/oidc/token',
  'api_key':'xhjheSC7AhSLtvapSDnbyFn17uWUqW5ccAOuHhQxnnEY',
  'resource_instance_id':'crn:v1:staging:public:cloud-object-storage:global:a/68a66698d275aeb48097f868957ab2ed:bbb5aa36-5525-4000-b129-bcb780195098::',
  'url':'https://s3-api.us-geo.objectstorage.uat.service.networklayer.com',
    'endpoint':'https://s3.us-west.objectstorage.uat.softlayer.net'
}
cos_credentials_prod_ki = {
  'iam_url':'https://iam.ng.bluemix.net/oidc/token',
  'api_key':'0s-JWmaDBwiSd_yWJqenoKRBfTVU5Rgkz31CDT5WgoWQ',
  'resource_instance_id':'crn:v1:bluemix:public:cloud-object-storage:global:a/db0d062d2b4c0836e18618a5222d8068:22e3b946-6154-4032-8e8f-7cfb0b429602::',
  'url':'https://s3-api.us-geo.objectstorage.service.networklayer.com',
      "endpoint":"https://s3-api.us-geo.objectstorage.softlayer.net",
}

# The Message Hub credentials should come from the 'Service credentials' page.

mh_credentials_stage1_1z = {
  "instance_id": "86ce5bfa-268e-4786-a7e9-91f8427f2e17",
  "mqlight_lookup_url": "https://mqlight-lookup-stage1.messagehub.services.us-south.bluemix.net/Lookup?serviceId=86ce5bfa-268e-4786-a7e9-91f8427f2e17",
  "api_key": "CBa3cYnfW1iWM25LNewJVZaxEP1qjKlThEeqz2Zrnkzu7VXd",
  "kafka_admin_url": "https://kafka-admin-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_rest_url": "https://kafka-rest-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_brokers_sasl": [
    "kafka02-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka04-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka01-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka03-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-stage1.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "CBa3cYnfW1iWM25L",
  "password": "NewJVZaxEP1qjKlThEeqz2Zrnkzu7VXd"
}
mh_credentials_stage1_2s = {
  "instance_id": "81b7462e-7707-44c1-8bfa-8c9490ac8111",
  "mqlight_lookup_url": "https://mqlight-lookup-stage1.messagehub.services.us-south.bluemix.net/Lookup?serviceId=81b7462e-7707-44c1-8bfa-8c9490ac8111",
  "api_key": "phXq2H0NSDQNSCdKGJrEFTSnVHjgH8ugpChw1LgNbL3Sr23g",
  "kafka_admin_url": "https://kafka-admin-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_rest_url": "https://kafka-rest-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_brokers_sasl": [
    "kafka04-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka03-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka01-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka02-stage1.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "phXq2H0NSDQNSCdK",
  "password": "GJrEFTSnVHjgH8ugpChw1LgNbL3Sr23g"
}

In [3]:
# Choose the credentials to use.

#cos_credentials = cos_credentials_stage1_wd
cos_credentials = cos_credentials_prod_ki
#mh_credentials = mh_credentials_stage1_1z
mh_credentials = mh_credentials_stage1_2s

In [4]:
# Load the yelp data

import watson_streaming_pipelines as stp

bucket_name = 'pyml'
object_name = 'yelp_academic_dataset_review_first200k.json'
#object_name = 'yelp_academic_dataset_review_first1k_training.json'

# TEST TEST
#object_name = 'test.txt'
#data_in = 'this is a test'
#resp = stp.put_to_cloud_object_storage(
#            api_key = cos_credentials['api_key'],
#            full_object_path = bucket_name + '/' + object_name, 
#            my_data = data_in,
#            auth_endpoint = cos_credentials['iam_url'],
#            service_endpoint = cos_credentials['endpoint']
#)
#print("resp = ", resp)

data = stp.get_from_cloud_object_storage( 
            api_key = cos_credentials['api_key'],
            full_object_path = bucket_name + '/' + object_name, 
            auth_endpoint = cos_credentials['iam_url'],
            service_endpoint = cos_credentials['endpoint']
) 
print(len(data))

174072961


In [5]:
# Convert and decode the binary data and load the yelp List with the JSON objects

from io import StringIO
import json

yelp = []
for line in StringIO(data.decode("utf-8")):
    yelp.append( json.loads(line))
print(yelp[0]['text'])

dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.


In [6]:
# Create the Message Hub producer
producer = stp.create_messagehub_producer( username = mh_credentials['user'], password = mh_credentials['password'], kafka_brokers_sasl = mh_credentials['kafka_brokers_sasl'])
mh_topic = 'testTopic1'

In [13]:
# Send selected yelp fields to the MH topic

import time
print(len(yelp))
#print(yelp[0]['stars'])
n=0
for msg in yelp[:40000]:
#    print(msg['stars'])
    producer.send( mh_topic, { 'stars': msg['stars'], 'text': msg['text']})
    n += 1
    if (n % 1000) == 0:
        time.sleep(1)
        print(n, "", end="")

200000
1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 